In [ ]:
'''Trains a simple network on the MNIST dataset to infer essential neurons for digit recognition.'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model 
from keras.layers import Input, Dense, Lambda, Flatten
from keras import backend as K
import numpy as np
from keras import optimizers
from keras import regularizers
import matplotlib.pyplot as plt
from copy import deepcopy
import seaborn as sns

In [ ]:
# %matplotlib notebook      #uncommment for interactive plotting

In [ ]:
# Data Pre-Processing

# input image dimensions
img_rows, img_cols = 28, 28

num_classes = 10

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# less data
## x_train = x_train[:15000]
## x_test = x_test[:3000]
## y_train = y_train[:15000]
## y_test = y_test[:3000]

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
import pickle
# Getting back the objects:
with open('../data/mnist_dataset.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    x_train, x_test, y_train, y_test = pickle.load(f)

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

## Neural Network Model and Training
### ***Training might take quite a bit of time. You can scroll down and load already trained network for further analysis. For clearity, the neural network structure and training details cell kept below.***

In [ ]:
# # MODEL Network w/ regularization on activations

# input_img = Input(shape=(784,))
# x = Dense(100, activation='sigmoid',activity_regularizer=regularizers.l1(0.04))(input_img)
# x = Dense(100, activation='sigmoid',activity_regularizer=regularizers.l1(0.04))(x)
# x = Dense(100, activation='sigmoid',activity_regularizer=regularizers.l1(0.04))(x)
# x = Dense(100, activation='sigmoid',activity_regularizer=regularizers.l1(0.04))(x)
# output_class = Dense(10, activation='softmax')(x)

# model = Model(input_img, output_class)

In [ ]:
#optimizer
# model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Training - manual
# batch_size = 1
# epochs = 30
# model.fit(x_train, y_train,
#           batch_size=batch_size,
#           epochs=epochs,
#           verbose=1,
#           validation_data=(x_test, y_test))

# score = model.evaluate(x_test, y_test, verbose=0)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])

In [ ]:
# score = model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', score[0])
# print('Test accuracy:', score[1])
# testacc = deepcopy(score[1])

In [ ]:
# model.save_weights('my_weights.model')

In [ ]:
# model.load_weights('my_weights.model')

In [ ]:
## SAVE MODEL
# model.save('my_model.h5')

## Load Trained Network for Analysis

In [ ]:
# LOAD MODEL
from keras.models import load_model
model = load_model('../data/model_sparseFinal.h5')
model.save_weights('my_weights.model')

In [ ]:
model.summary()

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
testacc = deepcopy(score[1])
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Finding Key Neurons via Compressed Sensing

### Finding Ground Truth with Single Neuron Ablations

In [ ]:
# mutate single neuron each time by assigning -100 bias to it. layer2
# my_weights_trainedFinal4 use this
Pground=[]
for i in range(100):
    model.load_weights('my_weights.model') #load saved model weights
    tempw = model.get_weights() 
    tempw[3][i] = -100
    model.set_weights(tempw)
    score = model.evaluate(x_test, y_test, verbose=0)
    Pground.append(score[1])
    

In [ ]:
Pground = np.array(Pground)
plt.plot(testacc - Pground)
plt.xlabel('Neurons', fontdict=None, labelpad=None)
plt.ylabel('Accuracy Drop', fontdict=None, labelpad=None)
plt.show()

### Compressed Sensing to find key neurons
For results in the paper, load the measurement matrix below instead of generating another random measurement matrix below.

In [ ]:
# mutate single neuron each time by assigning -100 bias to it.
# my_weights_trainedFinal4 use this
Mtemp = []
P=[]
N=50
for i in range(N):
    model.load_weights('my_weights.model') #load saved model weights
    tempw = model.get_weights()
    tempindex = np.random.randint(0, high=100, size=5)
    Mtemp.append(tempindex)
    tempw[3][tempindex] = -100
    model.set_weights(tempw)
    score = model.evaluate(x_test, y_test, verbose=0)
    P.append(score[1])
    

In [ ]:
P = np.array(P)
plt.barh(range(1,51),np.flip(testacc - P, axis=0))
plt.ylabel('Ablations', fontdict=None, labelpad=None)
plt.xlabel('Accuracy Drop - Group Phenotype', fontdict=None, labelpad=None)
plt.grid()
plt.show()

In [ ]:
# Make Measurement Matrix

M = np.zeros([N,100])
for i in range(N):
    M[i][Mtemp[i]] = 1.0

### Load the saved Measurement Matrix in the paper

In [ ]:
import pickle
# Getting back the objects:
with open('../data/MeasurementMatrix_P_Pground.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    M, P, Pground = pickle.load(f)

In [ ]:
#Solve LASSO
from sklearn.linear_model import Lasso

Pnew = testacc-P
P_infer_list = []
L1norm=[]
L2norm=[]
L12tot=[]

for k in range(100):
    alp = np.logspace(-4, -1.5, num=100)
    clf = Lasso(alpha=alp[k])
    clf.fit(M, Pnew)
    P_infer = clf.coef_
    P_infer_list.append(P_infer)
    L1norm.append(np.sum(np.abs(P_infer)))
    L2norm.append(np.sum((np.dot(M,P_infer)-Pnew)**2))
    L12tot.append(L2norm[-1] + alp[k]*L1norm[-1])

P_infer_array = np.asarray(P_infer_list)

In [ ]:
plt.plot(np.log10(alp),L2norm)
plt.show()

In [ ]:
for k in range(100):
    plt.plot(np.log10(alp),P_infer_array[:,k])
plt.show()


In [ ]:
plt.imshow(M, cmap='gray')
plt.show()

In [ ]:
Pground = np.array(Pground)
plt.plot(range(1,101), testacc - Pground)
plt.plot(range(1,101), -P_infer_array[40])
plt.show()

### Statistical Analysis of ANN

In [ ]:
### ********** THIS SIMULATIONS MIGHT TAKE HOURS ********** 
### *** For results in the paper, you can load the saved results below and run further cells.

# Msize=[25, 30, 35, 40, 45, 50, 55, 60]
# Pinfer_Msize=[];
# for k in Msize:
#     print('---'+str(k)+'---')
#     P_infer_temp_list=[]
#     for j in range(0,100):
        
#         Mtemp = []
#         P=[]
#         N=k
#         for i in range(N):
#             model.load_weights('my_weights.model') #load saved model weights
#             tempw = model.get_weights()
#             tempindex = np.random.randint(0, high=100, size=np.random.randint(5,10))
#             Mtemp.append(tempindex)
#             tempw[3][tempindex] = -100
#             model.set_weights(tempw)
#             score = model.evaluate(x_test, y_test, verbose=0)
#             P.append(score[1])

#         P = np.array(P)
#         # Make Measurement Matrix
#         M = np.zeros([N,100])
#         for i in range(N):
#             M[i][Mtemp[i]] = 1.0


#         #Solve LASSO
#         Pnew = testacc-P
#         clf = Lasso(alpha=0.001)
#         clf.fit(M, Pnew)
#         P_infer = clf.coef_
#         P_infer_temp_list.append(P_infer)
        
#         print(j)


#     P_infer_array = np.asarray(P_infer_temp_list)
#     Pinfer_Msize.append(P_infer_array)

### Load Saved Simulations

In [ ]:
import pickle
# Getting back the objects:
with open('../data/Pinfer_Msize.pkl', 'rb') as f:  # Python 3: open(..., 'rb')
    Pinfer_Msize = pickle.load(f)
    Pinfer_Msize=Pinfer_Msize[0]

### Find False Neg & Pos

In [ ]:
Pground2=testacc - Pground
Pground2_top7 = deepcopy(Pground2)

In [ ]:
keyunits7 = np.argsort(Pground2)[-7:]

In [ ]:
Pground2_top7[keyunits7]=1.0
Pground2_top7[Pground2_top7!=1.0]=0.

In [ ]:
falseNeg_Msize=[]
falsePos_Msize=[]
Msize=[25, 30, 35, 40, 45, 50, 55, 60]

for k in range(0,8):

    falseNeg=[]
    falsePos=[]
    for i in range(100):
        a = Pinfer_Msize[k][i,:]
        a = np.select([a == 0., a != 0.], [np.zeros_like(a), np.ones_like(a)])
        b = a - Pground2_top7
        falseNeg.append(np.count_nonzero(b == -1))
        falsePos.append(np.count_nonzero(b == 1))

    falseNeg = np.asarray(falseNeg)
    falsePos = np.asarray(falsePos)

    falseNeg_Msize.append(falseNeg)
    falsePos_Msize.append(falsePos)
    
falseNeg_Msize = np.asarray(falseNeg_Msize)
falsePos_Msize = np.asarray(falsePos_Msize)



In [ ]:
plt.errorbar(Msize,falsePos_Msize.mean(axis=1),falsePos_Msize.std(axis=1),capsize=4)
plt.errorbar(Msize,falseNeg_Msize.mean(axis=1),falseNeg_Msize.std(axis=1),capsize=4)
plt.ylabel('# of false neg & pos')
plt.xlabel('# of group measurements')
plt.legend(['False Pos', 'False Neg'])

### Visualize Activations

In [ ]:
def get_activations(model, layer, X_batch):
    get_activations = K.function([model.layers[0].input, K.learning_phase()], [model.layers[layer].output])
    activations = get_activations([X_batch,0])
    return activations

In [ ]:
y_labels = [np.argmax(y, axis=None, out=None) for y in y_test]
y_labels = np.array(y_labels)

In [ ]:
x_random = np.random.rand(10000,784)

In [ ]:
itemindex = np.where(y_labels==0) #EDIT HERE for different digits
itemindex = itemindex[0]

In [ ]:
# Get Activations
# if you want to get activation of a single image instead of batch of images, you need to reshape it.
# here is the explanation (https://stackoverflow.com/questions/40430186/tensorflow-valueerror-cannot-feed-value-of-shape-64-64-3-for-tensor-uplace)

#test_x_for_act = test_x.reshape(1,len(test_x))
#[101,126,136,148,157] - [4,6,19,24,27]

#firing = get_activations(model, 2, x_random)
firing = get_activations(model, 2, x_test)
firing = firing[0]

In [ ]:
#plt.plot(np.mean(firing,axis=0))
plt.plot(firing[472])
plt.xlabel('Neurons', fontdict=None, labelpad=None)
plt.ylabel('Firing Rate', fontdict=None, labelpad=None)
plt.show()